In [14]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
client = OpenAI()

vector_store = client.beta.vector_stores.create(
  name="GRITAE2"
)
print(vector_store)

VectorStore(id='vs_0yhzHPvJzcQT66OJ23HtNfRp', created_at=1717766228, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1717766228, metadata={}, name='GRITAE2', object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)


In [16]:
vector_store_files = client.beta.vector_stores.files.list(
  vector_store_id="vs_VwMZZPLE2n0cmHr0oqSgFzGL"
)
print(vector_store_files)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-MUUy46T4HmjR1ylBCit5vpF5', created_at=1717765253, last_error=None, object='vector_store.file', status='completed', usage_bytes=653708, vector_store_id='vs_VwMZZPLE2n0cmHr0oqSgFzGL', chunking_strategy={'type': 'static', 'static': {'max_chunk_size_tokens': 800, 'chunk_overlap_tokens': 400}}), VectorStoreFile(id='file-MyyxS1VHkBQPQuJ6THPYH94S', created_at=1717765252, last_error=None, object='vector_store.file', status='completed', usage_bytes=720855, vector_store_id='vs_VwMZZPLE2n0cmHr0oqSgFzGL', chunking_strategy={'type': 'static', 'static': {'max_chunk_size_tokens': 800, 'chunk_overlap_tokens': 400}}), VectorStoreFile(id='file-DBWXLsIg8aNax7I7e8sfsq4l', created_at=1717765251, last_error=None, object='vector_store.file', status='completed', usage_bytes=692515, vector_store_id='vs_VwMZZPLE2n0cmHr0oqSgFzGL', chunking_strategy={'type': 'static', 'static': {'max_chunk_size_tokens': 800, 'chunk_overlap_tokens': 400}}), VectorStore

In [18]:
len(list(vector_store_files))

30

In [2]:
from openai import OpenAI
client = OpenAI()

my_assistant = client.beta.assistants.create(
    instructions="You have been given HTML documents of case studies of projects done for different companies in the past."
    " You will be provided the about page of a website of a company that deals in a specific field of IT servicing."
    " You are to provide the name of the company that corresponds to the most similar case study, "
    "and also 3 reasons why that case study is worth going through by the prospect that is, how it could be beneficial for them."
    " in json format only. " 
    "Example output: {\"url\":\"www.ABC.com\", \"points\":[\"We have helped ABC do X more efficiently which also could benefit you as you deal in similar fields\", \"We incorporated X into the ABC ecosystem... \", \" We helped ABC do this better, you could use it too.....\"]}",
    name="GRITAE",
    tools=[{"type": "file_search"}],
    model="gpt-4o", 
    tool_resources={
    "file_search": {
      "vector_store_ids": ["vs_VwMZZPLE2n0cmHr0oqSgFzGL"]
    },
    }
)
print(my_assistant)

Assistant(id='asst_O2HWobGq5XTAMxMJCt0bMiMT', created_at=1717830712, description=None, instructions='You have been given HTML documents of case studies of projects done for different companies in the past. You will be provided the about page of a website of a company that deals in a specific field of IT servicing. You are to provide the name of the company that corresponds to the most similar case study, and also 3 reasons why that case study is worth going through by the prospect that is, how it could be beneficial for them. in json format only. Example output: {"url":"www.ABC.com", "points":["We have helped ABC do X more efficiently which also could benefit you as you deal in similar fields", "We incorporated X into the ABC ecosystem... ", " We helped ABC do this better, you could use it too....."]}', metadata={}, model='gpt-4o', name='GRITAE', object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_in

In [21]:
from openai import OpenAI
client = OpenAI()

my_assistant = client.beta.assistants.create(
    instructions="",
    name="GRITAE2",
    model="gpt-4o",
)
print(my_assistant)

Assistant(id='asst_EdCxX673x1nFzLPZhlD3yUZW', created_at=1717768602, description=None, instructions='', metadata={}, model='gpt-4o', name='GRITAE2', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)


In [22]:
thread = client.beta.threads.create()

In [ ]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id="asst_EdCxX673x1nFzLPZhlD3yUZW",
    model="gpt-4o",
    instructions="You will be given a list of endpoints of a particular website. You are to provide back the website link that is most likely to be the about page of the website. Return nothing but the link. The list is as follows: "+str()
)

In [4]:
import re


def extract_json_substring(text):
    # Define the regex pattern to match content between ```json and ```, excluding the markers
    pattern = r'```json\s*(.*?)\s*```'
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        # Extract and return the substring
        return match.group(1).strip()
    else:
        # Return None if no match is found
        return None

In [1]:
from openai import OpenAI
client = OpenAI()

my_assistant = client.beta.assistants.create(
    instructions="You have been given HTML documents of case studies of projects done for different companies in the past."
    " You will be provided the about page of a website of a company that deals in a specific field of IT servicing."
    " You are to provide the name of the company that corresponds to the most similar case study, "
    "and also 3 reasons why that case study is worth going through by the prospect that is, how it could be beneficial for them."
    " in json format only. " 
    "Example output: {\"url\":\"www.ABC.com\", \"points\":[\"We have helped ABC do X more efficiently which also could benefit you as you deal in similar fields\", \"We incorporated X into the ABC ecosystem... \", \" We helped ABC do this better, you could use it too.....\"]}",
    name="GRITAE",
    tools=[{"type": "file_search"}],
    model="gpt-4o", 
    tool_resources={
    "file_search": {
      "vector_store_ids": ["vs_VwMZZPLE2n0cmHr0oqSgFzGL"]
    },
    }
)

In [8]:
summary = "Loops is a company that develops an email platform designed for early-stage B2B SaaS companies, helping them manage email campaigns, automated onboarding, retention, and reengagement emails for their customers. Backed by Y Combinator, the company is focused on rapidly building new features and prides itself on consistently delivering quality work despite being a small team."

In [9]:

client = OpenAI()
thread = client.beta.threads.create()
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=my_assistant.id,
    model="gpt-4o",
    instructions="You have been given HTML documents of case studies for companies done for different companies in the past."
    "I will provide the summary of a company. Find the most similar case study that fits the needs of this company and provide 3 reasons why the case study is worth looking into."
    "Provide in json format only and nothing extra." 
    "Example output: {\"case_study\":\"Fundrise\", \"case_study_link\":\"https://www.intercom.com/customers/fundrise\", \"company_link\":\"https://fundrise.com/\"  \"reasons\":[\"We have helped ABC do X more efficiently which also could benefit you because...\", \"We incorporated X into the ABC ecosystem... \", \" We helped ABC do this better, you could use it too.....\"]}"
    "The summary of the company is as follows: " + summary
    )

In [10]:
import json
import re


if run.status == "completed":
        messages = client.beta.threads.messages.list(
            thread_id=thread.id,
            run_id=run.id
        )
        text = messages.data[0].content[0].text.value
        text = extract_json_substring(text)
        text = re.sub(r'【.*?】', '', text)
        text = text.replace("```json", "").replace("```","")
        text = json.loads(text)
        print(text)

{'case_study': 'WebinarGeek', 'case_study_link': 'https://www.intercom.com/customers/webinargeek', 'company_link': 'https://www.webinargeek.com/', 'reasons': ["WebinarGeek started as an early-stage company and used Intercom's Early Stage Program to transform their customer communications, which aligns with your current stage and need for a robust yet scalable solution.", "They saw a significant 35% increase in trial-to-paid conversion rates by using personalized and targeted email campaigns via Intercom's Series feature. This could be particularly valuable for Loops' focus on automated onboarding and engagement.", 'The consolidation of their tech stack was crucial for WebinarGeek, enabling them to manage support, sales, and marketing seamlessly. This would benefit Loops by integrating various functions within a single platform.']}


In [7]:
import requests
from bs4 import BeautifulSoup

# Function to fetch HTML content from a URL and extract text with line breaks
def extract_text_with_line_breaks_from_url(url):
    # Fetch the HTML content from the URL
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    html_content = response.text

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract text with proper line breaks
    text_with_line_breaks = '. '.join(list(set(soup.stripped_strings)))
    

    return text_with_line_breaks

# URL of the web page
url = 'https://www.loops.so/about'

# Get the text from the URL
text = extract_text_with_line_breaks_from_url(url)

# Print the extracted text
print(text)


Company. Engineering. as well as great founders from companies like. Terms of Service. That’s it! Have a great day 👋. our open roles. Send your first email. Send transactional email. We're the small but mighty team behind Loops.. Log in. company, backed by. View all. Lattice. and more.. We’re building an email platform for early-stage B2B SaaS companies.. Dan Rowden. Chris. Kyle Suss. Chris Frantz. Adam Kaczmarek. © 2024 Astrodon Inc.. We've been lucky to stand on the shoulders of giants. We're a. Product. Focus. Public Wiki. Hey there!. Codecademy. Framer. Changelog. Dropbox. by the way, one of the two founders of Loops. We’re a small team but we try to put out great work consistently. If you'd like to join us on our mission, check out. Resources. ,. Credits. Build a waitlist. I’m. Bounce Doctor. Pricing. Privacy. We take care of sending email campaigns for your product updates as well as automated onboarding, retention, and reengagement emails. We've got even more planned so stay tun

In [6]:
import tempfile
from openai import OpenAI
import re

client = OpenAI()

def save_html_to_tempfile(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.html', mode='w', encoding='utf-8')
        temp_file.write(response.text)
        temp_file.close()
        print(f'Successfully saved {url} to {temp_file.name}')
        return temp_file.name
    except requests.exceptions.RequestException as e:
        print(f'Failed to retrieve {url}: {e}')
        return None

url = "https://www.loops.so/about"
temp_file_path = save_html_to_tempfile(url)
print(temp_file_path)
vector_store = client.beta.vector_stores.create(
  name="testing"
)
cli =  client.files.create(file=open(temp_file_path, "rb"),purpose="assistants")
vector_store_file = client.beta.vector_stores.files.create(
        vector_store_id=vector_store.id,
        file_id=cli.id
        )
my_assistant = client.beta.assistants.create(
    instructions="You are a company summarizer. you have access to the about page of the website of the company",
    name="testing",
    model="gpt-4o",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)
thread = client.beta.threads.create()
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=my_assistant.id,
    model="gpt-4o",
    instructions="Provide a brief single paragraph summary on what the company does"
    )
if run.status == "completed":
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        run_id=run.id
    )
    text = messages.data[0].content[0].text.value
    #text = extract_json_substring(text)
    text = re.sub(r'【.*?】', '', text)
    #text = text.replace("```json", "").replace("```","")
    #text = json.loads(text)
    client.beta.vector_stores.delete(
        vector_store_id=vector_store.id
    )
    client.beta.assistants.delete(assistant_id=my_assistant.id)
    client.files.delete(file_id=cli.id)
    client.beta.threads.delete(thread_id=thread.id)
    print(text)

Successfully saved https://www.loops.so/about to /tmp/tmp8bjiifv_.html
/tmp/tmp8bjiifv_.html
The company specializes in providing risk management solutions and services, which include risk assessment, mitigation strategies, and implementation support to help clients manage and minimize potential risks in their operations .


In [11]:
my_assistant = client.beta.assistants.create(
    instructions="You are a company goals summarizer",
    name="testing",
    model="gpt-4o",
)
thread = client.beta.threads.create()
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=my_assistant.id,
    model="gpt-4o",
    instructions="Provide a brief single paragraph summary on what the following company does given the following sentences in their 'about' page: "+ text
    )
if run.status == "completed":
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        run_id=run.id
    )
    text = messages.data[0].content[0].text.value
    #text = extract_json_substring(text)
    text = re.sub(r'【.*?】', '', text)
    #text = text.replace("```json", "").replace("```","")
    #text = json.loads(text)
    client.beta.assistants.delete(assistant_id=my_assistant.id)
    client.beta.threads.delete(thread_id=thread.id)
    print(text)

Loops is an email platform designed specifically for early-stage B2B SaaS companies, helping them manage product update campaigns and automate emails aimed at onboarding, retaining, and reengaging customers. Supported by notable investors like Lattice and Y Combinator, Loops enhances customer communication strategies for SaaS businesses.


In [17]:
from xml.etree import ElementTree as ET

def get_single_nested_links(sitemap_url):
    """
    Retrieve links from a sitemap.xml with a maximum of single nesting.
    
    :param sitemap_url: URL of the sitemap.xml
    :return: List of filtered URLs
    """
    
    if sitemap_url[-1]!="/":
        sitemap_url += "/"
    if not sitemap_url.startswith("https://"):
        sitemap_url = "https://" + sitemap_url
        
    sitemap_url += "sitemap.xml"

    response = requests.get(sitemap_url)
    if response.status_code != 200:
        raise Exception(f"Failed to retrieve sitemap: {response.status_code}")

    sitemap_content = response.content
    root = ET.fromstring(sitemap_content)
    namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

    single_nested_links = []
    
    for url in root.findall('ns:url', namespaces=namespace):
        loc = url.find('ns:loc', namespaces=namespace).text
        count = loc.count("/")
        if count<7:
            single_nested_links.append(loc)

    return single_nested_links

In [23]:
links = get_single_nested_links("https://www.loops.so/")

assistant = client.beta.assistants.create(
  instructions="You are a website analyzer. You will provide the about page endpoint from a list of endpoints. Do not return anything but the endpoint and do not return it as a link",
  model="gpt-4o",
  tools=[
    {
      "type": "function",
      "function": {
        "name": "get_about_page",
        "description": "Get the link that is most likely to be the about page of the website",
        "parameters": {
          "type": "object",
          "properties": {
            "links": {
              "type": "array",
              "description": "The list of endpoints of a website",
              "items":{
                "type":"string"
              }
            }
          },
          "required": ["links"]
        }
      }
    }
  ]
)
thread = client.beta.threads.create()
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Which one is the about page for the following set of endpoints: "+str(links)
)
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)
tool_outputs = []
 
# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "get_about_page":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": "https://www.cognizant.com/us/en/about-cognizant"
    })

if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)

client.beta.assistants.delete(assistant_id=assistant.id)
client.beta.threads.delete(thread_id=thread.id)

requires_action
Tool outputs submitted successfully.
The about page for the given set of endpoints is [https://loops.so/about](https://loops.so/about).


ThreadDeleted(id='thread_JZ1LxMNWrj01irhDGImlFf1N', deleted=True, object='thread.deleted')

In [25]:
from urllib.parse import urlparse

def filter_endpoints(urls):
    def is_singly_nested(path):
        parts = path.strip('/').split('/')
        print(parts)
        return len(parts) < 2 and all(len(part) < 4 for part in parts)

    result = []
    for url in urls:
        parsed_url = urlparse(url)
        path = parsed_url.path
        if is_singly_nested(path):
            result.append(url)
    return result

# Example usage:
urls = [
    "http://example.com/abc/de",
    "http://example.com/abcd/de",
    "http://example.com/abc/defg",
    "http://example.com/abc/def",
    "http://example.com/abc",
    "http://example.com/abc/def/ghi",
    "http://example.com/ab/cd/ef",
    "http://example.com/us/en/about-us",
    "http://example.com/us/en/xyz"
]

filtered_urls = filter_endpoints(urls)
print(filtered_urls)


['abc', 'de']
['abcd', 'de']
['abc', 'defg']
['abc', 'def']
['abc']
['abc', 'def', 'ghi']
['ab', 'cd', 'ef']
['us', 'en', 'about-us']
['us', 'en', 'xyz']
['http://example.com/abc/def/ghi', 'http://example.com/ab/cd/ef', 'http://example.com/us/en/xyz']
